## 💾데이터 전처리 실습_2: 
**❗목표:** 월별 방문자 데이터를 범용적으로 분석 가능한 Long 형태로 변환

##### [원본: 04_중앙박물관_24-25년_월별_관람객](../data/raw/museum/csv/03_중앙박물관_7개년_관람객.csv)
##### [정제: museum_visitors_2425](../data/processed/museum_visitors_2425.csv)

## Step 1: 라이브러리 import & 파일 읽기

In [35]:
import pandas as pd

# 파일 읽기
df = pd.read_csv(r'C:\Users\Comet\Desktop\kpdh\data\raw\museum\csv\04_중앙박물관_24-25년_월별_관람객.csv')

# 원본 확인
print("원본 데이터:")
display(df)
print(f"\n원본 shape: {df.shape}")

원본 데이터:


,월,2024_총계,2024_내국인,2024_외국인,2025_총계,2025_내국인,2025_외국인
0,1월,325127,317286,7841,513262,505282,7980
1,2월,305525,296323,9202,543361,536035,7326
2,3월,222265,205603,16662,341471,325928,15543
3,4월,226589,205456,21133,348114,324329,23785
4,5월,286491,265843,20648,495384,472680,22704
5,6월,283530,264058,19472,474731,454084,20647
6,7월,361493,339913,21580,747679,720949,26730
7,8월,428217,408391,19826,864977,842049,22928
8,9월,351733,333975,17758,409025,384647,24378
9,10월,310040,288462,21578,553584,525901,27683



원본 shape: (13, 7)


## Step 2: 합계 행 제거

In [36]:
# 합계 행 제거 전 확인
print("제거 전 마지막 행:")
display(df.tail(2))

# 합계 행 제거
df = df[df['월'] != '합계'].copy()

print("\n제거 후 마지막 행:")
display(df.tail(2))
print(f"제거 후 shape: {df.shape}")

제거 전 마지막 행:


,월,2024_총계,2024_내국인,2024_외국인,2025_총계,2025_내국인,2025_외국인
11,12월,422114,412872,9242,693218,680668,12550
12,합계,3788785,3590700,198085,6507483,6276291,231192



제거 후 마지막 행:


,월,2024_총계,2024_내국인,2024_외국인,2025_총계,2025_내국인,2025_외국인
10,11월,265661,252518,13143,522677,503739,18938
11,12월,422114,412872,9242,693218,680668,12550


제거 후 shape: (12, 7)


## Step 3: 총계 컬럼 확인 & 제거

In [37]:
# 현재 컬럼 확인
print("현재 컬럼 목록:")
print(df.columns.tolist())

현재 컬럼 목록:
['월', '2024_총계', '2024_내국인', '2024_외국인', '2025_총계', '2025_내국인', '2025_외국인']


In [38]:
# 총계 컬럼 제거
df = df.drop(columns=['2024_총계', '2025_총계'])

print("\n총계 제거 후 컬럼:")
print(df.columns.tolist())
display(df.head(3))


총계 제거 후 컬럼:
['월', '2024_내국인', '2024_외국인', '2025_내국인', '2025_외국인']


,월,2024_내국인,2024_외국인,2025_내국인,2025_외국인
0,1월,317286,7841,505282,7980
1,2월,296323,9202,536035,7326
2,3월,205603,16662,325928,15543


## Step 4: Wide → Long 변환

In [39]:
# Wide to Long 변환
df_long = pd.melt(
    df,
    id_vars=['월'],              # 월 컬럼을 기준으로
    var_name='구분',             # 컬럼명이 '구분'으로
    value_name='방문자수'        # 값이 '방문자수'로
)

print("Long 형태 변환 결과:")
display(df_long.head(10))
print(f"\nLong 형태 shape: {df_long.shape}")

Long 형태 변환 결과:


,월,구분,방문자수
0,1월,2024_내국인,317286
1,2월,2024_내국인,296323
2,3월,2024_내국인,205603
3,4월,2024_내국인,205456
4,5월,2024_내국인,265843
5,6월,2024_내국인,264058
6,7월,2024_내국인,339913
7,8월,2024_내국인,408391
8,9월,2024_내국인,333975
9,10월,2024_내국인,288462



Long 형태 shape: (48, 3)


In [40]:
# split 후 rename으로 컬럼명 변경
temp = df_long['구분'].str.split('_', expand=True)
temp = temp.rename(columns={0: '연도', 1: '카테고리'})

# 원본에 합치기
df = pd.concat([df_long, temp], axis=1)

# '구분' 컬럼 삭제
df = df.drop('구분', axis=1)

# 컬럼 순서 변경 (연도, 카테고리를 앞으로)
df = df[['연도', '월','카테고리','방문자수']]

display(df.style.format({'방문자수': '{:,}'}))  # 원본 유지

,연도,월,카테고리,방문자수
0,2024,1월,내국인,"317,286"
1,2024,2월,내국인,"296,323"
2,2024,3월,내국인,"205,603"
3,2024,4월,내국인,"205,456"
4,2024,5월,내국인,"265,843"
5,2024,6월,내국인,"264,058"
6,2024,7월,내국인,"339,913"
7,2024,8월,내국인,"408,391"
8,2024,9월,내국인,"333,975"
9,2024,10월,내국인,"288,462"


## Step 6: 데이터 타입 변환

In [41]:
# 현재 데이터 타입 확인
print("변환 전 데이터 타입:")
print(df.dtypes)

변환 전 데이터 타입:
연도      object
월       object
카테고리    object
방문자수     int64
dtype: object


In [42]:
# 숫자형으로 변환
df['방문자수'] = pd.to_numeric(df['방문자수'])
df['연도'] = df['연도'].astype(int)
df['월'] = df['월'].str.replace('월', '').astype(int)

In [43]:
print("\n변환 후 데이터 타입:")
print(df.dtypes)
print("\n변환 결과 확인:")
display(df.head(6))


변환 후 데이터 타입:
연도       int64
월        int64
카테고리    object
방문자수     int64
dtype: object

변환 결과 확인:


,연도,월,카테고리,방문자수
0,2024,1,내국인,317286
1,2024,2,내국인,296323
2,2024,3,내국인,205603
3,2024,4,내국인,205456
4,2024,5,내국인,265843
5,2024,6,내국인,264058


## Step 7: 날짜 컬럼 생성

In [ ]:
# 날짜 컬럼 생성
df['날짜']=pd.to_datetime(
    df['연도'].astype(str)+'-'+
    df['월'].astype(str).str.zfill(2) + '-01'
)
print("날짜 컬럼 생성 결과:")
display(df.head(20))
print(f"\n날짜 데이터 타입: {df['날짜'].dtype}")

날짜 컬럼 생성 결과:


,연도,월,카테고리,날짜,방문자수
0,2024,1,내국인,2024-01-01,317286
1,2024,1,외국인,2024-01-01,7841
2,2024,2,내국인,2024-02-01,296323
3,2024,2,외국인,2024-02-01,9202
4,2024,3,내국인,2024-03-01,205603
5,2024,3,외국인,2024-03-01,16662
6,2024,4,내국인,2024-04-01,205456
7,2024,4,외국인,2024-04-01,21133
8,2024,5,내국인,2024-05-01,265843
9,2024,5,외국인,2024-05-01,20648



날짜 데이터 타입: datetime64[ns]


## Step 8: 컬럼 순서 정리 & 정렬

In [48]:
df.dtypes

year                 int64
month                int64
category            object
date        datetime64[ns]
visitors             int64
dtype: object

In [50]:
# 컬럼명 변경 및 순서 정리

df = df.rename(columns={
    '연도': 'year',
    '월': 'month',
    '카테고리': 'category',
    '날짜': 'date',
    '방문자수': 'visitors'
})

display(df.head())

,year,month,category,date,visitors
0,2024,1,내국인,2024-01-01,317286
1,2024,1,외국인,2024-01-01,7841
2,2024,2,내국인,2024-02-01,296323
3,2024,2,외국인,2024-02-01,9202
4,2024,3,내국인,2024-03-01,205603


In [51]:

# 정렬
df = df.sort_values(['date', 'category'], ascending=[True, True]).reset_index(drop=True)

print("최종 결과 :")
display(df.style.format({'visitors': '{:,}'}))
print(f"\n최종 shape: {df.shape}")
print(f"카테고리 종류: {df['category'].unique()}")

최종 결과 :


,year,month,category,date,visitors
0,2024,1,내국인,2024-01-01 00:00:00,"317,286"
1,2024,1,외국인,2024-01-01 00:00:00,"7,841"
2,2024,2,내국인,2024-02-01 00:00:00,"296,323"
3,2024,2,외국인,2024-02-01 00:00:00,"9,202"
4,2024,3,내국인,2024-03-01 00:00:00,"205,603"
5,2024,3,외국인,2024-03-01 00:00:00,"16,662"
6,2024,4,내국인,2024-04-01 00:00:00,"205,456"
7,2024,4,외국인,2024-04-01 00:00:00,"21,133"
8,2024,5,내국인,2024-05-01 00:00:00,"265,843"
9,2024,5,외국인,2024-05-01 00:00:00,"20,648"



최종 shape: (48, 5)
카테고리 종류: ['내국인' '외국인']


## Step 9: 기본 확인 (결측치, 통계)

In [ ]:
# 결측치 확인
print("결측치 개수:")
print(df.isnull().sum())

# 기본 통계
print("\n방문자수 기본 통계:")
display(df['방문자수'].describe())

# 카테고리별 합계
print("\n카테고리별 총 방문자수:")
display(df.groupby('카테고리')['방문자수'].sum())
df.index.name = 'id'
display(df)

In [ ]:
df.to_csv(r'C:\Users\Comet\Desktop\kpdh\data\processed\museum_visitors_2425.csv', 
          encoding='utf-8-sig', 
          index=True)